In [1]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

In [2]:
def mesh_fem_2d(xl, xr, yl, yr, Mx, My, k):
	tmp = np.array([[np.arange(0,k*Mx,k) + (k*Mx+1)*i*k] for i in range(My)]).flatten()
	tmp2 = np.array([[np.arange(k,k*Mx+1,k) + (k*Mx+1)*(i+1)*k] for i in range(My)]).flatten()
	tmp3 = list(np.concatenate([list(j*(Mx*k+1)+np.arange(0,k+1-j)) for j in range(k+1)]))
	ind4e = np.array([tmp[np.int32(i/2)]+tmp3 if i % 2 == 0 else tmp2[np.int32((i-1)/2)] - tmp3 for i in range(2*Mx*My)])
	n4e = np.array([[ind4e[i,k], ind4e[i,np.int32((k+1)*(k+2)/2-1)], ind4e[i,0]] for i in range(ind4e.shape[0])])
	n4db = np.concatenate([[i for i in range(0,k*Mx+1)], [i for i in range(2*k*Mx+1,(k*Mx+1)*(k*My+1),(k*Mx+1))],
			[i for i in range((k*Mx+1)*(k*My+1)-2,k*My*(k*Mx+1)-1,-1)], [i for i in range((k*My-1)*(k*Mx+1),k*Mx,-(k*Mx+1))]])
	x = np.linspace(xl,xr,k*Mx+1)
	y = np.linspace(yl,yr,k*My+1)
	x = np.tile(x, (1,k*My+1)).flatten()
	y = np.tile(y,(k*Mx+1,1)).T.flatten()
	c4n = np.array([[x[i], y[i]] for i in range(len(x))])
	return (c4n,n4e,n4db,ind4e)

**[Exercise 1]** Add the matrices for the cubic approximations ($k=3$) in **get_matrices_2d**

In [3]:
def get_matrices_2d_triangle(k=1):
	if k == 1:
		M_R = np.array([[2, 1, 1], [1, 2, 1], [1, 1, 2]])/6.
		Srr_R = np.array([[1, -1, 0], [-1, 1, 0], [0, 0, 0]])/2.
		Srs_R = np.array([[1, 0, -1], [-1, 0, 1], [0, 0, 0]])/2.
		Ssr_R = np.array([[1, -1, 0], [0, 0, 0], [-1, 1, 0]])/2.
		Sss_R = np.array([[1, 0, -1], [0, 0, 0], [-1, 0, 1]])/2.
		Dr_R = np.array([[-1, 1, 0], [-1, 1, 0], [-1, 1, 0]])/2.
		Ds_R = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])/2.
	elif k == 2:
		M_R = np.array([[6, 0, -1, 0, -4, -1], [0, 32, 0, 16, 16, -4], [-1, 0, 6, -4, 0, -1], 
			[0, 16, -4, 32, 16, 0], [-4, 16, 0, 16, 32, 0], [-1, -4, -1, 0, 0, 6]])/90.
		Srr_R = np.array([[3 ,-4, 1, 0, 0, 0], [-4, 8, -4, 0, 0, 0], [1, -4, 3, 0, 0, 0], 
			[0, 0, 0, 8, -8, 0], [0, 0, 0, -8, 8, 0], [0, 0, 0, 0, 0, 0]])/6.
		Srs_R = np.array([[3, 0, 0, -4, 0, 1], [-4, 4, 0, 4, -4, 0], [1, -4, 0, 0, 4, -1], 
			[0, 4, 0, 4, -4, -4], [0, -4, 0, -4, 4, 4], [0, 0, 0, 0, 0, 0]])/6.
		Ssr_R = np.array([[3, -4, 1, 0, 0, 0], [0, 4, -4, 4, -4, 0], [0, 0, 0, 0, 0, 0], 
			[-4, 4, 0, 4, -4, 0], [0, -4, 4, -4, 4, 0], [1, 0, -1, -4, 4, 0]])/6.
		Sss_R = np.array([[3, 0, 0, -4, 0, 1], [0, 8, 0, 0, -8, 0], [0, 0, 0, 0, 0, 0], 
			[-4, 0, 0, 8, 0, -4], [0, -8, 0, 0, 8, 0], [1, 0, 0, -4, 0, 3]])/6.
		Dr_R = np.array([[-3, 4, -1, 0, 0, 0], [-1, 0, 1, 0, 0, 0], [1, -4, 3, 0, 0, 0], 
			[-1, 2, -1, -2, 2, 0], [1, -2, 1, -2, 2, 0], [1, 0, -1, -4, 4, 0]])/2.
		Ds_R = np.array([[-3, 0, 0, 4, 0, -1], [-1, -2, 0, 2, 2, -1], [1, -4, 0, 0, 4, -1], 
			[-1, 0, 0, 0, 0, 1], [1, -2, 0, -2, 2, 1], [1, 0, 0, -4, 0, 3]])/2.

	###################################################################################################
	# TODO: Add the matrices for the cubic approximations ($k=3$).
	
	###################################################################################################		
	else:
		M_R = 0
		Srr_R = 0
		Srs_R = 0
		Ssr_R = 0
		Sss_R = 0
		Dr_R = 0
		Ds_R = 0
	return (M_R, Srr_R, Srs_R, Ssr_R, Sss_R, Dr_R, Ds_R)

In [4]:
def fem_for_poisson_2d(c4n,n4e,n4db,ind4e,M_R,Srr_R,Srs_R,Ssr_R,Sss_R,f,u_D):
	number_of_nodes = c4n.shape[0]
	number_of_elements = n4e.shape[0]
	b = np.zeros(number_of_nodes)
	u = np.zeros(number_of_nodes)
	xr = np.array([(c4n[n4e[i,0],0] - c4n[n4e[i,2],0])/2. for i in range(number_of_elements)])
	yr = np.array([(c4n[n4e[i,0],1] - c4n[n4e[i,2],1])/2. for i in range(number_of_elements)])
	xs = np.array([(c4n[n4e[i,1],0] - c4n[n4e[i,2],0])/2. for i in range(number_of_elements)])
	ys = np.array([(c4n[n4e[i,1],1] - c4n[n4e[i,2],1])/2. for i in range(number_of_elements)])
	J = xr*ys - xs*yr
	rx=ys/J
	ry=-xs/J
	sx=-yr/J
	sy=xr/J
	Aloc = np.array([J[i]*((rx[i]**2+ry[i]**2)*Srr_R.flatten() + (rx[i]*sx[i]+ry[i]*sy[i])*(Srs_R.flatten()+Ssr_R.flatten()) 
		+ (sx[i]**2+sy[i]**2)*Sss_R.flatten()) for i in range(number_of_elements)])
	for i in range(number_of_elements):
		b[ind4e[i]] += J[i]*np.matmul(M_R, f(c4n[ind4e[i]]))
	row_ind = np.tile(ind4e.flatten(),(ind4e.shape[1],1)).T.flatten()
	col_ind = np.tile(ind4e,(1,ind4e.shape[1])).flatten()
	A_COO = coo_matrix((Aloc.flatten(), (row_ind, col_ind)), shape=(number_of_nodes, number_of_nodes))
	A = A_COO.tocsr()
	dof = np.setdiff1d(range(0,number_of_nodes), n4db)
	u[dof] = spsolve(A[dof, :].tocsc()[:, dof].tocsr(), b[dof])
	return u

In [5]:
def compute_error_fem_2d(c4n,n4e,ind4e,M_R,Dr_R,Ds_R,u,ux,uy):
	error = 0
	number_of_elements = n4e.shape[0]
	xr = np.array([(c4n[n4e[i,0],0] - c4n[n4e[i,2],0])/2. for i in range(number_of_elements)])
	yr = np.array([(c4n[n4e[i,0],1] - c4n[n4e[i,2],1])/2. for i in range(number_of_elements)])
	xs = np.array([(c4n[n4e[i,1],0] - c4n[n4e[i,2],0])/2. for i in range(number_of_elements)])
	ys = np.array([(c4n[n4e[i,1],1] - c4n[n4e[i,2],1])/2. for i in range(number_of_elements)])
	J = xr*ys-xs*yr
	rx=ys/J
	ry=-xs/J
	sx=-yr/J
	sy=xr/J
	for i in range(number_of_elements):
		Du_x = np.matmul(rx[i]*Dr_R+sx[i]*Ds_R, u[ind4e[i]])
		Du_y = np.matmul(ry[i]*Dr_R+sy[i]*Ds_R, u[ind4e[i]])
		Dex = ux(c4n[ind4e[i]]) - Du_x
		Dey = uy(c4n[ind4e[i]]) - Du_y
		error += J[i] * (np.matmul(Dex,np.matmul(M_R,Dex)) + np.matmul(Dey,np.matmul(M_R,Dey)))
	return np.sqrt(error)

In [8]:
iter = 6
xl, xr, yl, yr=0, 1, 0, 1
M = 2 ** np.arange(2,iter+2)
f = lambda x: 2 * np.pi**2 * np.sin(np.pi * x[:,0]) * np.sin(np.pi * x[:,1])
u_D = lambda x: 0 * x[:,0]
ux = lambda x: np.pi * np.cos(np.pi * x[:,0]) * np.sin(np.pi * x[:,1])
uy = lambda x: np.pi * np.sin(np.pi * x[:,0]) * np.cos(np.pi * x[:,1])

h = 1 / M
k = 2
error = np.zeros(iter)
M_R, Srr_R, Srs_R, Ssr_R, Sss_R, Dr_R, Ds_R = get_matrices_2d_triangle(k)
for i in range(iter):
	c4n, n4e, n4db, ind4e = mesh_fem_2d(xl, xr, yl, yr, M[i], M[i], k)
	u = fem_for_poisson_2d(c4n,n4e,n4db,ind4e,M_R,Srr_R,Srs_R,Ssr_R,Sss_R,f,u_D)
	error[i] = compute_error_fem_2d(c4n,n4e,ind4e,M_R,Dr_R,Ds_R,u,ux,uy)

rate = (np.log(error[1:])-np.log(error[:-1]))/(np.log(h[1:])-np.log(h[:-1]))
print(rate)

[1.93804243 1.98287353 1.99552345 1.99885786 1.9997117 ]


**[Exercise 2]** Modify **fem_for_poisson_2d_triangle_ex2** to solve the Poisson problem with non-homogeneous Dirichlet boundary condition,
\begin{align*}
		-\Delta u(\boldsymbol{x}) &= \ f(\boldsymbol{x}) \qquad \textrm{in } \Omega \\
		u(\boldsymbol{x}) &= u_D(\boldsymbol{x}) \quad \ \ \  \textrm{on } \partial\Omega. \\
\end{align*}

In [ ]:
def fem_for_poisson_2d_ex2(c4n,n4e,n4db,ind4e,M_R,Srr_R,Srs_R,Ssr_R,Sss_R,f,u_D):
	number_of_nodes = c4n.shape[0]
	number_of_elements = n4e.shape[0]
	b = np.zeros(number_of_nodes)
	u = np.zeros(number_of_nodes)
	xr = np.array([(c4n[n4e[i,0],0] - c4n[n4e[i,2],0])/2. for i in range(number_of_elements)])
	yr = np.array([(c4n[n4e[i,0],1] - c4n[n4e[i,2],1])/2. for i in range(number_of_elements)])
	xs = np.array([(c4n[n4e[i,1],0] - c4n[n4e[i,2],0])/2. for i in range(number_of_elements)])
	ys = np.array([(c4n[n4e[i,1],1] - c4n[n4e[i,2],1])/2. for i in range(number_of_elements)])
	J = xr*ys - xs*yr
	rx=ys/J
	ry=-xs/J
	sx=-yr/J
	sy=xr/J
	Aloc = np.array([J[i]*((rx[i]**2+ry[i]**2)*Srr_R.flatten() + (rx[i]*sx[i]+ry[i]*sy[i])*(Srs_R.flatten()+Ssr_R.flatten()) 
		+ (sx[i]**2+sy[i]**2)*Sss_R.flatten()) for i in range(number_of_elements)])
	for i in range(number_of_elements):
		b[ind4e[i]] += J[i]*np.matmul(M_R, f(c4n[ind4e[i]]))
	row_ind = np.tile(ind4e.flatten(),(ind4e.shape[1],1)).T.flatten()
	col_ind = np.tile(ind4e,(1,ind4e.shape[1])).flatten()
	A_COO = coo_matrix((Aloc.flatten(), (row_ind, col_ind)), shape=(number_of_nodes, number_of_nodes))
	A = A_COO.tocsr()
	dof = np.setdiff1d(range(0,number_of_nodes), n4db)

	###################################################################################################
	# TODO: Add a few lines to treat non-homogeneous Dirichlet boundary condition. 
	
	###################################################################################################

	u[dof] = spsolve(A[dof, :].tocsc()[:, dof].tocsr(), b[dof])
	return u

In [ ]:
iter = 6
xl, xr, yl, yr=0, 1, 0, 1
M = 2 ** np.arange(2,iter+2)
f = lambda x: 2 * np.pi**2 * np.sin(np.pi * x[:,0]) * np.sin(np.pi * x[:,1])
u_D = lambda x: x[:,0]
ux = lambda x: np.pi * np.cos(np.pi * x[:,0]) * np.sin(np.pi * x[:,1]) + 1
uy = lambda x: np.pi * np.sin(np.pi * x[:,0]) * np.cos(np.pi * x[:,1])

h = 1 / M
k = 2
error = np.zeros(iter)
M_R, Srr_R, Srs_R, Ssr_R, Sss_R, Dr_R, Ds_R = get_matrices_2d_triangle(k)
for i in range(iter):
	c4n, n4e, n4db, ind4e = mesh_fem_2d(xl, xr, yl, yr, M[i], M[i], k)
	u = fem_for_poisson_2d_ex2(c4n,n4e,n4db,ind4e,M_R,Srr_R,Srs_R,Ssr_R,Sss_R,f,u_D)
	error[i] = compute_error_fem_2d(c4n,n4e,ind4e,M_R,Dr_R,Ds_R,u,ux,uy)

rate = (np.log(error[1:])-np.log(error[:-1]))/(np.log(h[1:])-np.log(h[:-1]))
print(rate)

**[Exercise 3]** Modify **fem_for_poisson_2d_ex3** to solve the Poisson problem with mixed boundary condition,
\begin{align*}
		-\Delta u(\boldsymbol{x}) &= \ f(\boldsymbol{x}) \qquad \ \textrm{in } \Omega \\
		u(\boldsymbol{x}) &= u_D(\boldsymbol{x}) \qquad \textrm{on } \Gamma_D \\
		\nabla u(\boldsymbol{x})\cdot\boldsymbol{n} &= u_N(\boldsymbol{x}) \qquad \textrm{on } \Gamma_N,
\end{align*}
where $\Gamma_D$ denotes the Dirichlet boundary, $\Gamma_N$ denotes the Neumann boundary, and $\boldsymbol{n}$ is the outward unit normal vector. 

In [ ]:
def fem_for_poisson_2d_ex3(c4n,n4e,n4db,ind4nb,ind4e,M_R,Srr_R,Srs_R,Ssr_R,Sss_R,M_R1D,f,u_D):
	number_of_nodes = c4n.shape[0]
	number_of_elements = n4e.shape[0]
	b = np.zeros(number_of_nodes)
	u = np.zeros(number_of_nodes)
	xr = np.array([(c4n[n4e[i,0],0] - c4n[n4e[i,2],0])/2. for i in range(number_of_elements)])
	yr = np.array([(c4n[n4e[i,0],1] - c4n[n4e[i,2],1])/2. for i in range(number_of_elements)])
	xs = np.array([(c4n[n4e[i,1],0] - c4n[n4e[i,2],0])/2. for i in range(number_of_elements)])
	ys = np.array([(c4n[n4e[i,1],1] - c4n[n4e[i,2],1])/2. for i in range(number_of_elements)])
	J = xr*ys - xs*yr
	rx=ys/J
	ry=-xs/J
	sx=-yr/J
	sy=xr/J
	Aloc = np.array([J[i]*((rx[i]**2+ry[i]**2)*Srr_R.flatten() + (rx[i]*sx[i]+ry[i]*sy[i])*(Srs_R.flatten()+Ssr_R.flatten()) 
		+ (sx[i]**2+sy[i]**2)*Sss_R.flatten()) for i in range(number_of_elements)])
	for i in range(number_of_elements):
		b[ind4e[i]] += J[i]*np.matmul(M_R, f(c4n[ind4e[i]]))
	row_ind = np.tile(ind4e.flatten(),(ind4e.shape[1],1)).T.flatten()
	col_ind = np.tile(ind4e,(1,ind4e.shape[1])).flatten()
	A_COO = coo_matrix((Aloc.flatten(), (row_ind, col_ind)), shape=(number_of_nodes, number_of_nodes))
	A = A_COO.tocsr()
	dof = np.setdiff1d(range(0,number_of_nodes), n4db)

	###################################################################################################
	# TODO: Add a few lines to treat Neumann boundary condition.
	
	###################################################################################################

	###################################################################################################
	# TODO: Add a few lines to treat non-homogeneous Dirichlet boundary condition. 
	#       If you finished Exercise 2, the following lines can be copied from `fem_for_poisson_1d_ex2`
	
	###################################################################################################
	
	u[dof] = spsolve(A[dof, :].tocsc()[:, dof].tocsr(), b[dof])
	return u

In [ ]:
iter = 6
xl, xr, yl, yr=0, 1, 0, 1
M = 2 ** np.arange(2,iter+2)
f = lambda x: 2 * np.pi**2 * np.sin(np.pi * x[:,0]) * np.sin(np.pi * x[:,1])
u_D = lambda x: x[:,0]
u_N = lambda x: -np.pi * np.sin(np.pi * x[:,0]) * np.cos(np.pi * x[:,1])
ux = lambda x: np.pi * np.cos(np.pi * x[:,0]) * np.sin(np.pi * x[:,1]) + 1
uy = lambda x: np.pi * np.sin(np.pi * x[:,0]) * np.cos(np.pi * x[:,1])

h = 1 / M
k = 2
error = np.zeros(iter)
M_R, Srr_R, Srs_R, Ssr_R, Sss_R, Dr_R, Ds_R = get_matrices_2d_triangle(k)
from fem_1d import get_matrices_1d
M_R1D,_,_ = get_matrices_1d(k)
for i in range(iter):
	c4n, n4e, n4db, ind4e = mesh_fem_2d(xl, xr, yl, yr, M[i], M[i], k)
	ind4nb = np.array([list(np.arange(j*k,(j+1)*k+1)) for j in range(0,M[i])])
	n4db = np.concatenate((np.array([0,k*M[i]]), np.setdiff1d(n4db,ind4nb.flatten())),axis=0)
	u = fem_for_poisson_2d_ex3(c4n,n4e,n4db,ind4nb,ind4e,M_R,Srr_R,Srs_R,Ssr_R,Sss_R,M_R1D,f,u_D,u_N)
	error[i] = compute_error_fem_2d(c4n,n4e,ind4e,M_R,Dr_R,Ds_R,u,ux,uy)

rate = (np.log(error[1:])-np.log(error[:-1]))/(np.log(h[1:])-np.log(h[:-1]))
print(rate)